# Sample Calculations
below are the formulas/derivations/miscellaneous notes for the input parameters used in the sbr.py cantera model, along with demonstrations of the calculations for Turn over frequency, residence time, etc. 

## Reactor Specifications

### Reactor Volume

<img src="./images/spinning_basket_reactor_Graff.png" width=700 />

spinning basket reactor radius and volume specified in Graaf paper: 

$r = 35mm\pm0.5mm$  

$h = 70mm\pm0.5mm$

pp63 graaf, 1988 "The inner diameter and height of the reactor chamber were 70 mm"



Volume from grabow is 134.7E-6 m^3. this is pi\*r^2\*h/2, which I believe is because the baffles and machinery take up half of the reactor chamber but currently there is no justification for that to be found. 

In [13]:
import numpy as np
import scipy
volume = ((35e-3)**2)*np.pi*(70e-3)/2
print(volume)

0.0001346957850226624


### surface sites

Grabow took the average of the sites from this paper:  
Muhler, Cat. Lett. 14 (1992), 241

<img src="./images/input_calcs/sites_paper.png" width=700 />

the following (same paper) is a justification for the factor of 5: 

<img src="./images/input_calcs/sites_paper_2.png" width=700 />

In [14]:
import uncertainties as unc  
import uncertainties.unumpy as unumpy 

In [15]:
site_density = 5*61.67*1e-6*1e3 # [moles/kg]
site_density

0.30835

In [16]:
total_sites = site_density*4.24e-3 #moles sites (4.24 g cat)
total_sites

0.001307404

In [17]:
rmg_site_density_cu = 2.943e-9*1e4 #mol/m^2 see chemkin surface file
cat_area = (total_sites)/(rmg_site_density_cu) #mol/mol/m^2()
cat_area

44.4241930003398

 propagate error. use 7.97g for a conservate estimate to use for all readings (7.97 was the max weight used for catalyst)

In [18]:
sa_values = np.array([66,60,59])
sa_val = np.mean(sa_values)
# since we only have 3 values, conservative estimate is +/- 1/2 range
sa_unc = (max(sa_values) - min(sa_values))/2
site_density_u = unc.ufloat(sa_val, sa_unc)*5*1e-6*1e3
site_density_u

0.3083333333333333+/-0.0175

In [31]:
cat_weight = unc.ufloat(7.97e-3, 5e-4) #+/- 0.5 below instrument precision
total_sites_u = site_density_u*cat_weight
total_sites_u

0.0024574166666666663+/-0.00020789573525234015

In [32]:
rmg_site_density_cu = 2.943e-9*1e4 #mol/m^2 see chemkin surface file
cat_area_u = (total_sites_u)/(rmg_site_density_cu) #mol/mol/m^2()
cat_area_u

83.5003964208857+/-7.064075271910981

Reactor volume to cat surface area

In [22]:
vol_2_sa = volume / cat_area
vol_2_sa

3.0320367332645103e-06

## Inlet Conditions

### volume flowrate

It does not look like Graaf had a flowrate measurement on the inlet of their apparatus:
<img src="./images/input_calcs/flow_debacle.png" width=400 height=400 />


Further, in the Grabow matlab model, they use the volume (in cm^3/min) that grabow specifies as the outlet flowrate
<img src="./images/input_calcs/matlab_flow_1.png" width=600/>
<img src="./images/input_calcs/matlab_flow_2.png" width=600/>

for ours, we'll copy grabow, although realistically the outlet volume flow could change depending on the stoichiometry: 
<img src="./images/input_calcs/fogler_flow.jpg" width=300/>

for CO2/(CO+CO2) ratio experiments, conditions were 75 atm, 528K, and inlet flow was 470.4 cm^3/min

$V_{in}\left[\frac{m^3}{s}\right] =  V_{in}\left[\frac{cm^3}{min}\right]*\left[\frac{min}{60s}\right]*\left[\frac{m^3}{10^6 cm^3}\right]$

In [23]:
Vin_cm3_min = 470.4
Vin_m3_sec = Vin_cm3_min/(60*10**6)
Vin_m3_sec

7.84e-06

Address uncertainty in Volume flow. we don't have any detail on this, so assume $\pm$ 1%. they used a soap bubble meter of unspecified range, so this seems like a reasonable estimate, see 10.1088/0950-7671/41/7/309, which says for 0-1000 c^3/s error is below 1%. we are at ~7.3 cm^3/s. 

In [29]:
470/60

7.833333333333333

In [30]:
Vin_cm3_min_u = unc.ufloat(470.4, 470.4*0.01)
Vin_m3_sec_u = Vin_cm3_min_u/(60*10**6)
Vin_m3_sec_u

7.84e-06+/-7.84e-08

#### pressure and temp
the pressure and temp were measured directly. no direct indication of error so just use 1 more than instrument uncertainty

In [ ]:
temp_u = unc.ufloat(500, 0.1*500)
press_u = unc.ufloat(500, 0.1*500)

#### mole fractions
the uncertainty in the mole fractions is not specified. we can derive molar flow uncertainty, but that will not tell us anything about mole fraction uncertainty, since they collected their gas as a liquid after and ran it through a gc. We can use the error in the material balances thay found to make a conservative estimate of error. they say it was usually below 0.5%, so we will assume error is ~1% of measurement. this will be applied for inlet and outlet.  
<img src="./images/Graaf_mole_frac_unc.png" width=600/>

### Turn over Frequency
For the Graaf data, Grabow just multiplied the total molar flowrate in by the mole fraction, and then divided by the number of sites

for cantera, our ROP is in terms of kmoles/m^2 s. so, to convert to 1/s, we need to do the following:

$ROP\left[\frac{kmoles}{m^{2}*s}\right]*\frac{10^{3} moles}{kmol} * \frac{1}{\Gamma} \left[\frac{cm^2}{mol}\right] * \frac{m^2}{10^{4}cm^{2}}= ROP \left[\frac{mol}{sec}\right]$   

$TOF = ROP \left[\frac{mol}{sec}\right]* \frac{1}{N_{sites}} \left[\frac{1}{moles}\right]$





# Other Miscellaneous notes on inputs

#### Pressure conversion 
Grabow data is in bar.  